### From the file now get the embeddings.... 
This is bascially now reproducing the results for zero-shot pre-trained model...
We first generate the embeddings from our pre-trained model and then get a cosine distance....

In [1]:
import warnings
from os import environ
from pathlib import Path

#import demo_utils
import numpy as np
import pandas as pd
import torch

from bmfm_targets import config
from bmfm_targets.datasets import dnaseq
from bmfm_targets.tasks import task_utils
from bmfm_targets.tokenization import load_tokenizer

warnings.filterwarnings("ignore")


ImportError: cannot import name '_disabled_torch_function_impl' from 'torch._C' (unknown location)

In [13]:
import os

print(os.getcwd())
# The pre-trained model...
checkpoint_path = "/dccstor/bmfm-targets/users/hongyang/training_runs/scbert_ref_2kb_change0.3/epoch=17-step=264312-val_loss=5.42.ckpt" #/dccstor/bmfm-targets/users/hongyang/training_runs/scbert_snp_2kb/epoch=19-step=293680-val_loss=5.32.ckpt
#checkpoint_path = "/proj/bmfm/users/hongyang/training_runs/ref_rc_1kb_10kb_10x_modernbert_v3/backup_ckpt/epoch=18-step=138282-val_loss=4.40.ckpt"

ckpt_dict = torch.load(checkpoint_path)

/dccstor/bmfm-targets1/users/sanjoy/bmfm-targets/bmfm_targets/datasets/dnaseq/preprocess_dataset


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

# Get the dataloader for this SNV_MPRA dataset...

In [6]:
# if "model_config" in ckpt_dict["hyper_parameters"]:
#     fields = [i for i in ckpt_dict["hyper_parameters"]["model_config"].fields if i.is_input]
# else:
#     fields = config.main_config.default_fields()
#     tokenizer = load_tokenizer(Path(checkpoint_path).parent)
#     for f in fields:
#         f.update_vocab_size(tokenizer)
#     processed_name = "processed"

In [7]:
# fields

[FieldInfo(field_name='dna_chunks', vocab_size=4096, pretrained_embedding=None, vocab_update_strategy='static', is_masked=True, is_input=True, masked_output_modes=['token_scores'], tokenization_strategy='tokenize', num_special_tokens=0, continuous_value_encoder_kwargs=None)]

In [ ]:
#from bmfm_targets.datasets import dnaseq

dataset_kwargs={"processed_data_source":"/dccstor/bmfm-targets1/data/omics/genome/snv_mpra_cagi_regression/data_imputed.csv", 
                    "dataset_name": "cagi_4096", 
                    #"processed_file_name": "processed_imputed_snp2vec",
                    #"label_dict_path" : "/dccstor/bmfm-targets/data/omics/genome/MPRA/LLM_eval/data/CAGI/4096/cagi_4096_all_labels.json"
                    }
print(dataset_kwargs)

# predict_dataset = dnaseq.DNASeqMPRADataset(
#                 **dataset_kwargs,
#                 split=None
#                 )
               

{'processed_data_source': '/dccstor/bmfm-targets1/data/omics/genome/snv_mpra_cagi_regression/data_imputed.csv', 'dataset_name': 'cagi_4096'}


In [8]:
label_columns = [
    config.LabelColumnInfo(
        label_column_name="dummy_label", n_unique_values=1, is_regression_label=True
    )
]



## Prepare configurations required for running prediction task

In [ ]:
def get_embeds(dataset_kwargs, checkpoint_path, label_columns):
    ckpt_dict = torch.load(checkpoint_path)
    if "model_config" in ckpt_dict["hyper_parameters"]:
        fields = [i for i in ckpt_dict["hyper_parameters"]["model_config"].fields if i.is_input]
    else:
        fields = config.main_config.default_fields()
        tokenizer = load_tokenizer(Path(checkpoint_path).parent)
        for f in fields:
            f.update_vocab_size(tokenizer)
        processed_name = "processed"


    data_module = dnaseq.DNASeqMPRADataModule(  
        tokenizer=load_tokenizer(Path(checkpoint_path).parent),
        fields = fields,
        max_length=512,
        label_columns=label_columns,
        collation_strategy="sequence_classification",
        batch_size = 32,
        dataset_kwargs=dataset_kwargs,
    )
    data_module.prepare_data()
    data_module.setup("predict")

    label_dict = data_module.label_dict[label_columns[0].label_column_name] # pylint: disable=unsubscriptable-object

    trainer_config = config.TrainerConfig(batch_size = 64, losses=None, pooling_method="first_token")

    task_config = config.PredictTaskConfig(
        default_root_dir = environ["BMFM_TARGETS_TRAINING_DIR"],    
        output_predictions= True,
        output_embeddings= True,
        checkpoint=checkpoint_path,
        )

    tokenizer = load_tokenizer(Path(checkpoint_path).parent)

    if "model_config" in ckpt_dict["hyper_parameters"]:
        model_config = ckpt_dict["hyper_parameters"]["model_config"]
        model_config.checkpoint = checkpoint_path
        model_config.num_labels=len(label_dict)
    else:
        model_config = config.SCBertConfig(
        checkpoint= checkpoint_path,
        fields=fields,
        num_labels=len(label_dict),
        classifier_dropout=0,
        hidden_dropout_prob=0,
        attention_probs_dropout_prob=0,
        )

    main = config.main_config.SCBertMainConfig(data_module=data_module,task=task_config, model=model_config, trainer=trainer_config, fields=fields)
    main.complete_config()

    pl_module = task_utils.instantiate_module_from_checkpoint(
            main.task, main.data_module, main.model, main.trainer
        )
    results = task_utils.predict(pl_module, data_module, task_config)
    embeds = results['embeddings']

    cos = []
    dot = []
    l1 = []
    l2 = []
    N = int(len(embeds)/2)

    for a in range(N):
        ref_out = embeds[a]
        alt_out = embeds[a+N]
        cos.append((ref_out * alt_out).sum()/(np.linalg.norm(ref_out)*np.linalg.norm(alt_out)))
        dot.append((ref_out * alt_out).sum())
        l1.append(np.absolute(ref_out - alt_out).sum())
        l2.append(np.square(ref_out - alt_out).sum())

    cagi_result = {'cos': cos, 'dot': dot, 'l1': l1, 'l2': l2}

    return cagi_result

In [ ]:

cagi_result = get_embeds(dataset_kwargs, checkpoint_path, label_columns)

#cagi_result = h5py.File('../data/CAGI/'+'cagi_'+datalen+'_'+'dnabert2.h5', 'r')

['TERT-GBM' 'IRF6' 'GP1BB' 'HBB' 'PKLR' 'SORT1' 'MSMB' 'MYCrs6983267'
 'ZFAND3' 'HNF4A' 'TERT-HEK293T' 'HBG1' 'LDLR' 'IRF4' 'F9']


In [ ]:
cagi_df = pd.read_csv('/dccstor/bmfm-targets1/users/sanjoy/tools/LLM_eval/data/CAGI/4096/final_cagi_metadata.csv',
                      index_col=0).reset_index()
target = cagi_df['6'].values.tolist()
exp_list = cagi_df['8'].values.tolist()
cagi_df.head()
print(cagi_df.shape)

(18442, 11)


In [ ]:
# This is for the ref_gen model....
import scipy.stats as stats

perf = []
for key in cagi_result.keys():
    print(key)
    cagi_llr = cagi_result[key]
    for exp in ['LDLR','SORT1','F9','PKLR']:
        sub_df = cagi_df[cagi_df['8'] == exp]
        exp_target = np.array(target)[sub_df.index.to_list()]
        exp_pred = np.squeeze(cagi_llr)[sub_df.index.to_list()]
        exp_target = np.absolute(exp_target)
        exp_pred = np.absolute(exp_pred)
        print(exp)
        perf.append(stats.pearsonr(exp_pred,exp_target)[0])
        print(stats.pearsonr(exp_pred,exp_target)[0])

cos
LDLR
0.0015450893499268405
SORT1
-0.19116159127891397
F9
0.00254269970811529
PKLR
0.0983730996423263
dot
LDLR
0.1575734506587024
SORT1
-0.11913160366694353
F9
-0.20712877126917972
PKLR
-0.11220313065309032
l1
LDLR
0.013581015718045254
SORT1
0.20186603003820217
F9
3.1194853220864634e-05
PKLR
-0.09416090487376942
l2
LDLR
-0.0015822063988885022
SORT1
0.19111208313703
F9
-0.002713243433964822
PKLR
-0.09838357283979413
